***Chapter 3 Product and Profitability Analysis***

**Analysis Purpose**  
This chapter focuses on Walmart's product lines and customer structure, finding high-potential and low-efficiency categories through dual analysis of product sales and profits, and evaluating the overall profitability structure in combination with customer consumption levels.

In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path

In [ ]:
df = pd.read_excel("/Users/wendychu/Desktop/python/walmart ba project/my_env1/data/walmart_time.xlsx")

plt.rcParams["font.sans-serif"] = ["Heiti TC"]  # macOS # For normal display of Chinese labels
plt.rcParams['axes.unicode_minus'] = False 

**3.1 Sales and Profit Contribution by Product Category**

**Purpose**  
Identify the contribution of each product category to total sales and profits, finding abnormal combinations of "high sales but low profit" or "low sales but high profit".

**Method**  
- Use bar charts to show total sales, total profit, and average profit margin for each category  
- Compare profit structure differences between categories to discover imbalances

**Insights and Application Suggestions**  
- Fashion accessories and Home categories contribute the majority of sales, but their profit margins are not the highest  
- Some categories such as Food or Health have low sales and low profit margins, consider reducing exposure or integrating them as accessory items  


In [ ]:
df.head()
df['profit'] = df['total'] * df['profit_margin']
category_performance=df.groupby("category").agg({"total":"sum","quantity":"sum", "profit":"sum","rating":"mean","invoice_id":"nunique"}).reset_index()

# Calculate various metrics
# Profit margin
category_performance["profit_margin"]=category_performance["profit"]/category_performance["total"]*100
# Average transaction value
category_performance["each_transaction"]=category_performance["total"]/category_performance["invoice_id"]
# Number of items per order
category_performance["each_order_qua"]=category_performance["quantity"]/category_performance["invoice_id"]

category_performance.sort_values("profit_margin",ascending=False)



# Visualization
# Profit chart
plt.figure(figsize=(14,7))
ax1=plt.subplot(1,2,1)
sns.barplot(x="category",y="profit",data=category_performance,ax=ax1)
plt.title("Profit by Product Category")
plt.xlabel("Product Category")
plt.ylabel("Profit")
plt.xticks(rotation=45)
plt.grid(True,axis='y',linestyle='--', alpha=0.7)
for i,v in enumerate(category_performance["profit"]) :
    plt.text(i,v,f"{v:.0f}",ha="center")

# Profit margin chart

ax2=plt.subplot(1,2,2)
sns.barplot(x="category",y="profit_margin",data=category_performance,ax=ax2)
plt.title("Profit Margin by Product Category")
plt.xlabel("Product Category")
plt.ylabel("Profit Margin")
plt.xticks(rotation=45)
plt.grid(True,axis='y',linestyle='--', alpha=0.7)
for i,v in enumerate(category_performance["profit_margin"]) :
    plt.text(i,v,f"{v:.2f}",ha="center")

report=[]
report.append("Product Category Comprehensive Performance\n")


print("\nProduct Category Comprehensive Performance:")
report.append(category_performance[['category', 'total', 'profit', 'profit_margin', 
                          'each_transaction', 'each_order_qua', 'rating']].round(2).to_string(index=False))
report_text = "\n".join(report)
Path("Product_Category_Sales_Performance.txt").write_text(report_text,encoding="utf-8")
print(report_text)
plt.tight_layout()
plt.savefig('Product_Profit_Analysis.png', dpi=300)




In [ ]:
# 3.1 Product Mix Analysis
category_performance.head()
print(report_text)



**3.2 Product BCG Matrix Analysis**

**Purpose**  
From a product portfolio perspective, use the BCG matrix to visualize the strategic positioning of each product category, identifying which "stars" should receive focused resources and which "dogs" should be eliminated.

**Method**  
- Use monthly average sales growth rate (X-axis) and profit contribution (Y-axis) of product categories as judgment criteria  
- Divide products into four categories: cash cows, stars, question marks, and dogs, with bubble size representing sales amount

**Insights and Application Suggestions**  
- Cash cow categories (high profit but low growth) are stable revenue sources, recommend continuing to optimize the supply chain  
- Star categories should receive increased marketing resources (such as Fashion)  
- Question mark and dog categories should have their positioning and necessity of existence re-examined (such as Sports, Electronics)


In [ ]:
# BCG matrix chart classifying products as stars, niche, volume-driven, or problem

plt.figure(figsize=(12,10))

# Use sales percentage and profit margin as BCG dimensions

category_performance["sales_percentage"]=category_performance["total"]/(category_performance["total"].sum())*100
x=category_performance["sales_percentage"]
y=category_performance["profit_margin"]
size=category_performance["total"]/1000

# Create BCG matrix
plt.scatter(x,y,s=size,alpha=0.7,c=range(len(category_performance)),cmap='tab10')

# Create quadrant dividing lines
xm=x.mean()
ym=y.mean()
plt.axvline(x=xm,color="gray", linestyle='--')
plt.axhline(y=ym,color="gray", linestyle='--')

plt.text(xm + 2, ym + 0.2,"Star Products",fontsize=16)
plt.text(xm - 5, ym + 0.2,"Question Mark Products",fontsize=16)
plt.text(xm + 3, ym - 0.2, "Cash Cow Products", fontsize=16, ha='center')
plt.text(xm - 3, ym - 0.2, "Dog Products", fontsize=16, ha='center')


for i,v in enumerate(category_performance["category"]):
    plt.annotate(v,(x[i],y[i]),fontsize=9,xytext=(6, 4), textcoords='offset points')
plt.title('Product Category BCG Matrix Analysis', fontsize=16, fontweight='bold')
plt.xlabel('Sales Percentage (%)', fontsize=14)
plt.ylabel('Profit Margin', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig("Product_Category_BCG_Matrix_Analysis",dpi=300)



**3.3 Customer Segmentation: Consumption Levels and Profit Structure**

**Purpose**  
Analyze the contribution of different customer consumption levels (high, medium, low) to sales, profit, and ratings, evaluating customer value structure and loyalty potential.

**Method**  
- Divide customers into high, medium, and low tiers based on single transaction amount  
- Calculate total sales, average profit, and average rating for each tier  
- Compare satisfaction levels and repurchase potential across tiers

**Insights and Application Suggestions**  
- High-spending customers have high ratings and large profit contributions, making them the most valuable group to cultivate  
- Mid-tier customers have conversion potential, can introduce membership systems and upgrade recommendations  
- Low-tier customers have low purchase frequency and relatively low ratings, can improve experience through service improvements or low-price incentives

In [ ]:
# Originally planned to analyze sales by profit but found that profit margins don't vary much, so changed to analyzing consumer shopping behavior
df['transac_group'] = pd.qcut(df['total'], 4, labels=['Low Spending', 'Medium-Low Spending', 'Medium-High Spending', 'High Spending'])

# Calculate - total sales - total purchase quantity - total profit and average rating - number of orders
customer_performance=df.groupby("transac_group").agg({"total":"sum","quantity":"sum","profit":"sum","rating":"mean","invoice_id":"nunique"}).reset_index()

# Calculate - profit margin - average number of items per order
customer_performance["profit_margin"]=customer_performance["profit"]/customer_performance["total"]
customer_performance["per_order_quan"]=customer_performance["quantity"]/customer_performance["invoice_id"]

customer_performance.to_csv("Customer_Tier_Sales_Performance.csv", index=False, encoding="utf-8-sig")
print(customer_performance)
# Visualize profit margin and sales amount by different customer spending tiers
plt.figure(figsize=(14,10))
plt.subplot(2,1,1)
x=range(len(customer_performance))
width=0.35
plt.bar(x,customer_performance["total"],width,label="Sales")
plt.bar([i+width for i in x],customer_performance["profit"],width,label="Profit")

plt.title("Sales and Profit by Customer Spending Tier")
plt.xticks([i + width/2 for i in x],customer_performance["transac_group"])
plt.xlabel("Customer Spending Tier")
plt.ylabel("Amount")
plt.legend(loc="upper left")
plt.grid(True,axis="y",linestyle="--",alpha=0.7)

for i,v in enumerate(customer_performance["total"]):
    plt.text(i,v+5000,f'{v:.0f}',ha="center")

for i,v in enumerate(customer_performance["profit"]):
    plt.text(i+width,v+5000,f'{v:.0f}',ha="center")



# Visualize product quantity and satisfaction by different customer spending tiers
plt.subplot(2,1,2)
ax1=plt.gca()
ax2=ax1.twinx()


x=range(len(customer_performance))
width=0.35
ax1.bar(x,customer_performance["quantity"],width,label="Product Quantity")
ax1.set_ylabel("Product Quantity", color="blue")

ax2.plot(x,customer_performance["rating"],'ro-',linewidth=2,label="Satisfaction")  
ax2.set_ylabel("Average Rating", color="red")

for i,v in enumerate(customer_performance["quantity"]):
    ax1.text(i,v,f'{v:.0f}',ha="center")

plt.title("Product Quantity and Satisfaction by Customer Spending Tier")
plt.xticks([i for i in x],customer_performance["transac_group"])
ax1.set_xlabel("Customer Spending Tier")
plt.grid(True,axis="y",linestyle="--",alpha=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

plt.tight_layout()
plt.savefig("Customer_Tier_Calculation",dpi=300)


**3.4 Customer Purchase Behavior: Average Transaction Value and Purchase Quantity Analysis**

**Purpose**  
Observe the purchasing methods of different customer groups: whether they tend toward "low quantity, high price" or "high variety, low price", to plan product mix and promotional strategies.

**Method**  
- Create a scatter plot analysis of average transaction amount and average number of items purchased  
- Explore the correlation between the two and the purchasing habits of different customer types

**Insights and Application Suggestions**  
- Customer consumption behavior shows clear segmentation, with both high-price/few-categories and many-categories/low-price segments existing  

In [ ]:
# Categorize from a product category perspective

sorted_by_transaction = category_performance.sort_values('each_transaction', ascending=False)

plt.figure(figsize=(14,10))
ax1=plt.gca()
ax2=ax1.twinx()
width=0.35
ax1.bar(sorted_by_transaction["category"],sorted_by_transaction["each_transaction"],width,label="Average Transaction Value")
ax1.set_ylabel("Average Transaction Value", color="blue")
ax1.set_xlabel("Product Category")
ax1.set_title("Comparison of Average Transaction Value and Items Per Order by Product Category",fontsize=14)

ax2.plot(sorted_by_transaction["category"],sorted_by_transaction["each_order_qua"],"ro-",label="Items Per Order")  
ax2.set_ylabel("Items Per Order", color="red")

for i,v in enumerate(sorted_by_transaction["each_transaction"]):
    ax1.text(i,v,f'{v:.0f}',ha="center")

plt.grid(True,axis="y",linestyle="--",alpha=0.7)


line1,label1=ax1.get_legend_handles_labels()
line2,label2=ax2.get_legend_handles_labels()
ax1.legend(line1 + line2, label1 + label2)


plt.tight_layout()
plt.savefig("Comparison_of_Average_Transaction_Value_and_Items_Per_Order_by_Product_Category",dpi=300)




In [ ]:
sorted_by_transaction.head()